In [ ]:
import numpy as np
import pandas as pd

In [29]:
import csv

# Function to generate the output for each row
def generate_color_reference(name, red, green, blue):
    return f'new ColorReference() {{ Name="{name}", Argb=new Vector3({red},{green},{blue}) }},'

# Path to your CSV file
csv_file = 'colors.csv'

# Read the CSV file and process each row
with open(csv_file, encoding="utf8") as file:
    reader = csv.reader(file)
    next(reader)  # Skip the header if there is one
    for row in reader:
        color_name, red, green, blue = row
        print(generate_color_reference(color_name, red, green, blue))

new ColorReference() { Name="parakeet", Argb=new Vector3(174,182,87) },
new ColorReference() { Name="saddle brown", Argb=new Vector3(88,52,1) },
new ColorReference() { Name="cucumber crush", Argb=new Vector3(222,237,215) },
new ColorReference() { Name="pool blue", Argb=new Vector3(134,194,201) },
new ColorReference() { Name="distance", Argb=new Vector3(98,110,130) },
new ColorReference() { Name="light urple", Argb=new Vector3(179,111,246) },
new ColorReference() { Name="east side", Argb=new Vector3(172,145,206) },
new ColorReference() { Name="florida seashells", Argb=new Vector3(250,228,199) },
new ColorReference() { Name="paris", Argb=new Vector3(145,167,189) },
new ColorReference() { Name="powder blush", Argb=new Vector3(247,215,215) },
new ColorReference() { Name="middle blue", Argb=new Vector3(126,212,230) },
new ColorReference() { Name="turtle green", Argb=new Vector3(165,171,139) },
new ColorReference() { Name="relaxing blue", Argb=new Vector3(137,156,171) },
new ColorReference()

In [3]:
import os
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


In [3]:
import torch
from PIL import Image
import requests
from transformers import CLIPProcessor, CLIPModel
import os

In [5]:
from PIL import Image
import requests
import torch
from transformers import AutoProcessor, AutoTokenizer, CLIPModel
import os

# Load the pre-trained CLIP model
model = CLIPModel.from_pretrained("openai/clip-vit-large-patch14").to("cuda" if torch.cuda.is_available() else "cpu")

# Get the text tokenizer and processor for images
tokenizer = AutoTokenizer.from_pretrained("openai/clip-vit-large-patch14")
processor = AutoProcessor.from_pretrained("openai/clip-vit-large-patch14")

# Define the text prompts for different colors
color_prompts = ["a red car", "a blue car", "a green car", "a yellow car", "a black car"]
inputs = tokenizer(color_prompts, padding=True, return_tensors="pt")

# Get the text features (color descriptions)
text_features = model.get_text_features(**inputs).to("cuda" if torch.cuda.is_available() else "cpu")
text_features = text_features / text_features.norm(dim=-1, keepdim=True)  # Normalize text features

# Folder where your red car images are stored
image_folder = "C:/Users/Asus/Downloads/BLUE"

# Initialize performance metrics
total_images = 0
correct_predictions = 0
ground_truth = "red"  # All images are red cars

# Iterate over each image in the folder
for image_file in os.listdir(image_folder):
    if image_file.endswith(('png', 'jpg', 'jpeg')):
        total_images += 1

        # Load and preprocess the image
        image_path = os.path.join(image_folder, image_file)
        image = Image.open(image_path)
        inputs = processor(images=image, return_tensors="pt")

        # Move inputs to the correct device (CPU/GPU)
        inputs = {k: v.to("cuda" if torch.cuda.is_available() else "cpu") for k, v in inputs.items()}

        # Get the image features
        image_features = model.get_image_features(**inputs)
        image_features = image_features / image_features.norm(dim=-1, keepdim=True)  # Normalize image features

        # Compute cosine similarity between image features and text features
        similarities = (image_features @ text_features.T).softmax(dim=-1)

        # Get the color with the highest similarity score
        predicted_color_idx = similarities.argmax().item()
        predicted_color = color_prompts[predicted_color_idx].split()[1]  # extract the color from the prompt

        # Check if the prediction is correct (since ground truth is "red")
        if predicted_color == ground_truth:
            correct_predictions += 1

# Calculate accuracy
accuracy = correct_predictions / total_images * 100
print(f"Accuracy: {accuracy:.2f}%")


c:\Users\Asus\Downloads\llm-main\llm-main\.venv\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Accuracy: 0.00%


In [11]:
from PIL import Image
import torch
from transformers import AutoProcessor, AutoTokenizer, CLIPModel
import os

# Load the pre-trained CLIP model
device = "cuda" if torch.cuda.is_available() else "cpu"
model = CLIPModel.from_pretrained("openai/clip-vit-large-patch14").to(device)

# Get the text tokenizer and processor for images
tokenizer = AutoTokenizer.from_pretrained("openai/clip-vit-large-patch14")
processor = AutoProcessor.from_pretrained("openai/clip-vit-large-patch14")

# Define the text prompts for different colors
color_prompts = ["a red car", "a blue car", "a green car", "a yellow car", "a black car"]
inputs = tokenizer(color_prompts, padding=True, return_tensors="pt").to(device)

# Get the text features (color descriptions)
text_features = model.get_text_features(**inputs)
text_features = text_features / text_features.norm(dim=-1, keepdim=True)  # Normalize text features

# Folder where your red car images are stored
image_folder = "BLUE"

# Initialize performance metrics
total_images = 0
correct_predictions = 0
ground_truth = "green"  # All images are red cars

# Iterate over each image in the folder
for image_file in os.listdir(image_folder):
    if image_file.endswith(('png', 'jpg', 'jpeg')):
        total_images += 1

        # Load and preprocess the image
        image_path = os.path.join(image_folder, image_file)
        image = Image.open(image_path)

        # Preprocess the image to match CLIP's expected format
        inputs = processor(images=image, return_tensors="pt").to(device)

        # Get the image features
        image_features = model.get_image_features(**inputs)
        image_features = image_features / image_features.norm(dim=-1, keepdim=True)  # Normalize image features

        # Compute cosine similarity between image features and text features (without softmax)
        similarities = (image_features @ text_features.T).squeeze(0)  # Remove batch dimension

        # Get the color with the highest similarity score
        predicted_color_idx = similarities.argmax().item()
        predicted_color = color_prompts[predicted_color_idx].split()[1]  # extract the color from the prompt

        # Check if the prediction is correct (since ground truth is "red")
        if predicted_color == ground_truth:
            correct_predictions += 1

# Calculate accuracy
accuracy = correct_predictions / total_images * 100
print(f"Accuracy: {accuracy:.2f}%")


Accuracy: 42.86%


In [14]:
from PIL import Image
import torch
from transformers import AutoProcessor, AutoTokenizer, CLIPModel
import os
from sklearn.metrics import confusion_matrix, classification_report

# Load the pre-trained CLIP model
device = "cuda" if torch.cuda.is_available() else "cpu"
model = CLIPModel.from_pretrained("openai/clip-vit-large-patch14").to(device)

# Get the text tokenizer and processor for images
tokenizer = AutoTokenizer.from_pretrained("openai/clip-vit-large-patch14")
processor = AutoProcessor.from_pretrained("openai/clip-vit-large-patch14")

# Define the text prompts for different colors
color_prompts = ["red car", "blue car", "green car", "yellow car", "black car"]
inputs = tokenizer(color_prompts, padding=True, return_tensors="pt").to(device)

# Get the text features (color descriptions)
text_features = model.get_text_features(**inputs)
text_features = text_features / text_features.norm(dim=-1, keepdim=True)  # Normalize text features

# Folder where your car images are stored
image_folder = "BLUE"  # Change this to the folder containing your images

# Ground truth (the correct color for the images in your dataset)
ground_truth_color = "blue car"

# Initialize performance metrics
total_images = 0
correct_predictions = 0

# Arrays to store the predicted and actual labels
y_true = []
y_pred = []

# Iterate over each image in the folder
for image_file in os.listdir(image_folder):
    if image_file.endswith(('png', 'jpg', 'jpeg')):
        total_images += 1

        # Load and preprocess the image
        image_path = os.path.join(image_folder, image_file)
        image = Image.open(image_path)

        # Preprocess the image to match CLIP's expected format
        inputs = processor(images=image, return_tensors="pt").to(device)

        # Get the image features
        image_features = model.get_image_features(**inputs)
        image_features = image_features / image_features.norm(dim=-1, keepdim=True)  # Normalize image features

        # Compute cosine similarity between image features and text features (without softmax)
        similarities = (image_features @ text_features.T).squeeze(0)  # Remove batch dimension

        # Get the color with the highest similarity score
        predicted_color_idx = similarities.argmax().item()
        predicted_color = color_prompts[predicted_color_idx].split()[1]  # extract the color from the prompt

        # Store the ground truth and predicted labels for evaluation
        y_true.append(ground_truth_color)
        y_pred.append(predicted_color)

        # Check if the prediction is correct (since ground truth is "green")
        if predicted_color == ground_truth_color:
            correct_predictions += 1

# Calculate accuracy
accuracy = correct_predictions / total_images * 100
print(f"Accuracy: {accuracy:.2f}%")

# Confusion Matrix and Classification Report
print("\nConfusion Matrix:")
conf_matrix = confusion_matrix(y_true, y_pred, labels=["red car", "blue car", "green car", "yellow car", "black car"])
print(conf_matrix)

print("\nClassification Report:")
class_report = classification_report(y_true, y_pred, target_names=["red car", "blue car", "green car", "yellow car", "black car"])
print(class_report)


c:\Users\Asus\Downloads\llm-main\llm-main\.venv\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Accuracy: 0.00%

Confusion Matrix:
[[0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]

Classification Report:


ValueError: Number of classes, 2, does not match size of target_names, 5. Try specifying the labels parameter

In [17]:
from PIL import Image
import torch
from transformers import AutoProcessor, AutoTokenizer, CLIPModel
import os
from sklearn.metrics import confusion_matrix, classification_report

# Load the pre-trained CLIP model
device = "cuda" if torch.cuda.is_available() else "cpu"
model = CLIPModel.from_pretrained("openai/clip-vit-large-patch14").to(device)

# Get the text tokenizer and processor for images
tokenizer = AutoTokenizer.from_pretrained("openai/clip-vit-large-patch14")
processor = AutoProcessor.from_pretrained("openai/clip-vit-large-patch14")

# Define the text prompts for different colors
color_prompts = ["a red car", "a blue car", "a green car", "a yellow car", "a black car"]
inputs = tokenizer(color_prompts, padding=True, return_tensors="pt").to(device)

# Get the text features (color descriptions)
text_features = model.get_text_features(**inputs)
text_features = text_features / text_features.norm(dim=-1, keepdim=True)  # Normalize text features

# Folder where your car images are stored
image_folder = "green"  # Change this to the folder containing your images

# Ground truth (the correct color for the images in your dataset)
ground_truth_color = "green"

# Initialize performance metrics
total_images = 0
correct_predictions = 0

# Arrays to store the predicted and actual labels
y_true = []
y_pred = []

# Iterate over each image in the folder
for image_file in os.listdir(image_folder):
    if image_file.endswith(('png', 'jpg', 'jpeg')):
        total_images += 1

        # Load and preprocess the image
        image_path = os.path.join(image_folder, image_file)
        image = Image.open(image_path)

        # Preprocess the image to match CLIP's expected format
        inputs = processor(images=image, return_tensors="pt").to(device)

        # Get the image features
        image_features = model.get_image_features(**inputs)
        image_features = image_features / image_features.norm(dim=-1, keepdim=True)  # Normalize image features

        # Compute cosine similarity between image features and text features (without softmax)
        similarities = (image_features @ text_features.T).squeeze(0)  # Remove batch dimension

        # Get the color with the highest similarity score
        predicted_color_idx = similarities.argmax().item()
        predicted_color = color_prompts[predicted_color_idx].split()[1]  # extract the color from the prompt

        # Store the ground truth and predicted labels for evaluation
        y_true.append(ground_truth_color)
        y_pred.append(predicted_color)

        # Check if the prediction is correct (since ground truth is "green")
        if predicted_color == ground_truth_color:
            correct_predictions += 1

# Calculate accuracy
accuracy = correct_predictions / total_images * 100
print(f"Accuracy: {accuracy:.2f}%")

# Get unique labels from ground truth and predictions
unique_labels = sorted(list(set(y_true + y_pred)))  # Dynamically detect classes present in data

# Confusion Matrix and Classification Report
print("\nConfusion Matrix:")
conf_matrix = confusion_matrix(y_true, y_pred, labels=unique_labels)
print(conf_matrix)

print("\nClassification Report:")
class_report = classification_report(y_true, y_pred, target_names=unique_labels, labels=unique_labels)
print(class_report)
print("Unique labels in ground truth:", set(y_true))
print("Unique labels in predictions:", set(y_pred))


c:\Users\Asus\Downloads\llm-main\llm-main\.venv\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Accuracy: 100.00%

Confusion Matrix:
[[23]]

Classification Report:
              precision    recall  f1-score   support

       green       1.00      1.00      1.00        23

    accuracy                           1.00        23
   macro avg       1.00      1.00      1.00        23
weighted avg       1.00      1.00      1.00        23

Unique labels in ground truth: {'green'}
Unique labels in predictions: {'green'}


c:\Users\Asus\Downloads\llm-main\llm-main\.venv\Lib\site-packages\sklearn\metrics\_classification.py:386: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(


In [18]:
#LLAVA MODEL

from PIL import Image
import torch
from transformers import AutoProcessor, AutoTokenizer, AutoModel
import os
from sklearn.metrics import confusion_matrix, classification_report

# Load the pre-trained LLaVA model
device = "cuda" if torch.cuda.is_available() else "cpu"
model = AutoModel.from_pretrained("llava-hf/llava-1.5-7b-hf").to(device)  # Replace with actual LLaVA model name
tokenizer = AutoTokenizer.from_pretrained("llava-hf/llava-1.5-7b-hf")  # Replace with actual LLaVA tokenizer name

# Define the text prompts for different colors
color_prompts = ["a red car", "a blue car", "a green car", "a yellow car", "a black car"]
inputs = tokenizer(color_prompts, padding=True, return_tensors="pt").to(device)

# Get the text features (color descriptions)
text_features = model.get_text_features(**inputs)
text_features = text_features / text_features.norm(dim=-1, keepdim=True)  # Normalize text features

# Folder where your car images are stored
image_folder = "green"  # Change this to the folder containing your images

# Ground truth (the correct color for the images in your dataset)
ground_truth_color = "green"

# Initialize performance metrics
total_images = 0
correct_predictions = 0

# Arrays to store the predicted and actual labels
y_true = []
y_pred = []

# Iterate over each image in the folder
for image_file in os.listdir(image_folder):
    if image_file.endswith(('png', 'jpg', 'jpeg')):
        total_images += 1

        # Load and preprocess the image
        image_path = os.path.join(image_folder, image_file)
        image = Image.open(image_path).convert("RGB")

        # Preprocess the image to match LLaVA's expected format
        image_input = tokenizer(images=image, return_tensors="pt").to(device)  # Adjust according to LLaVA's requirements

        # Get the image features
        image_features = model.get_image_features(**image_input)
        image_features = image_features / image_features.norm(dim=-1, keepdim=True)  # Normalize image features

        # Compute cosine similarity between image features and text features (without softmax)
        similarities = (image_features @ text_features.T).squeeze(0)  # Remove batch dimension

        # Get the color with the highest similarity score
        predicted_color_idx = similarities.argmax().item()
        predicted_color = color_prompts[predicted_color_idx].split()[1]  # extract the color from the prompt

        # Store the ground truth and predicted labels for evaluation
        y_true.append(ground_truth_color)
        y_pred.append(predicted_color)

        # Check if the prediction is correct (since ground truth is "green")
        if predicted_color == ground_truth_color:
            correct_predictions += 1

# Calculate accuracy
accuracy = correct_predictions / total_images * 100
print(f"Accuracy: {accuracy:.2f}%")

# Get unique labels from ground truth and predictions
unique_labels = sorted(set(color_prompts))  # Ensure all prompts are included

# Confusion Matrix and Classification Report
print("\nConfusion Matrix:")
conf_matrix = confusion_matrix(y_true, y_pred, labels=unique_labels)
print(conf_matrix)

print("\nClassification Report:")
class_report = classification_report(y_true, y_pred, target_names=unique_labels, labels=unique_labels)
print(class_report)


ValueError: Unrecognized configuration class <class 'transformers.models.llava.configuration_llava.LlavaConfig'> for this kind of AutoModel: AutoModel.
Model type should be one of AlbertConfig, AlignConfig, AltCLIPConfig, ASTConfig, AutoformerConfig, BarkConfig, BartConfig, BeitConfig, BertConfig, BertGenerationConfig, BigBirdConfig, BigBirdPegasusConfig, BioGptConfig, BitConfig, BlenderbotConfig, BlenderbotSmallConfig, BlipConfig, Blip2Config, BloomConfig, BridgeTowerConfig, BrosConfig, CamembertConfig, CanineConfig, ChameleonConfig, ChineseCLIPConfig, ChineseCLIPVisionConfig, ClapConfig, CLIPConfig, CLIPVisionConfig, CLIPSegConfig, ClvpConfig, LlamaConfig, CodeGenConfig, CohereConfig, ConditionalDetrConfig, ConvBertConfig, ConvNextConfig, ConvNextV2Config, CpmAntConfig, CTRLConfig, CvtConfig, Data2VecAudioConfig, Data2VecTextConfig, Data2VecVisionConfig, DbrxConfig, DebertaConfig, DebertaV2Config, DecisionTransformerConfig, DeformableDetrConfig, DeiTConfig, DetaConfig, DetrConfig, Di

In [19]:
from PIL import Image
import os
import torch
from transformers import AutoProcessor, AutoModelForCausalLM

# Load the LLaVA model and processor
processor = AutoProcessor.from_pretrained("llava-hf/llava-1.5-7b-hf")
model = AutoModelForCausalLM.from_pretrained("llava-hf/llava-1.5-7b-hf")

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# Define the folder containing your images
image_folder = "green"  # Replace with the path to your image folder

# Define the conversation template
def create_conversation(image_filename):
    return [
        {
            "role": "user",
            "content": [
                {"type": "image"},
                {"type": "text", "text": "What is the color of the car in this image?"},
            ],
        },
        {
            "role": "assistant",
            "content": [{"type": "text", "text": "The car is red."}],  # Placeholder, will be replaced by model output
        },
    ]

# Iterate over each image in the folder
for image_file in os.listdir(image_folder):
    if image_file.endswith(('png', 'jpg', 'jpeg')):
        image_path = os.path.join(image_folder, image_file)
        image = Image.open(image_path).convert("RGB")

        # Create conversation based on the current image
        conversation = create_conversation(image_file)

        # Format the conversation using the processor
        text_prompt = processor.apply_chat_template(conversation, add_generation_prompt=True)
        print(f"Formatted Prompt for {image_file}:", text_prompt)

        # Tokenize the formatted prompt
        inputs = processor(text=text_prompt, return_tensors="pt", padding=True, truncation=True).to(device)

        # Process the image
        image_inputs = processor(images=image, return_tensors="pt").to(device)

        # Generate the response
        with torch.no_grad():
            outputs = model.generate(**inputs, **image_inputs, max_length=50)
            
        # Decode the generated response
        response = processor.decode(outputs[0], skip_special_tokens=True)
        print(f"Generated Response for {image_file}:", response)


ValueError: Unrecognized configuration class <class 'transformers.models.llava.configuration_llava.LlavaConfig'> for this kind of AutoModel: AutoModelForCausalLM.
Model type should be one of BartConfig, BertConfig, BertGenerationConfig, BigBirdConfig, BigBirdPegasusConfig, BioGptConfig, BlenderbotConfig, BlenderbotSmallConfig, BloomConfig, CamembertConfig, LlamaConfig, CodeGenConfig, CohereConfig, CpmAntConfig, CTRLConfig, Data2VecTextConfig, DbrxConfig, ElectraConfig, ErnieConfig, FalconConfig, FuyuConfig, GemmaConfig, Gemma2Config, GitConfig, GPT2Config, GPT2Config, GPTBigCodeConfig, GPTNeoConfig, GPTNeoXConfig, GPTNeoXJapaneseConfig, GPTJConfig, JambaConfig, JetMoeConfig, LlamaConfig, MambaConfig, Mamba2Config, MarianConfig, MBartConfig, MegaConfig, MegatronBertConfig, MistralConfig, MixtralConfig, MptConfig, MusicgenConfig, MusicgenMelodyConfig, MvpConfig, NemotronConfig, OlmoConfig, OpenLlamaConfig, OpenAIGPTConfig, OPTConfig, PegasusConfig, PersimmonConfig, PhiConfig, Phi3Config,

In [21]:
from PIL import Image
import os
import torch
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics import confusion_matrix, classification_report

# Load the pre-trained LLaVA model and tokenizer
device = "cuda" if torch.cuda.is_available() else "cpu"
model = AutoModel.from_pretrained("llava-hf/llava-1.5-7b-hf").to(device)
tokenizer = AutoTokenizer.from_pretrained("llava-hf/llava-1.5-7b-hf")

# Define the text prompts for different colors
color_prompts = ["a red car", "a blue car", "a green car", "a yellow car", "a black car"]
inputs = tokenizer(color_prompts, padding=True, return_tensors="pt").to(device)

# Get the text features (color descriptions)
text_features = model.get_text_features(**inputs)
text_features = text_features / text_features.norm(dim=-1, keepdim=True)  # Normalize text features

# Folder where your car images are stored
image_folder = "green"  # Change this to the folder containing your images

# Ground truth (the correct color for the images in your dataset)
ground_truth_color = "green"

# Initialize performance metrics
total_images = 0
correct_predictions = 0

# Arrays to store the predicted and actual labels
y_true = []
y_pred = []

# Iterate over each image in the folder
for image_file in os.listdir(image_folder):
    if image_file.endswith(('png', 'jpg', 'jpeg')):
        total_images += 1

        # Load and preprocess the image
        image_path = os.path.join(image_folder, image_file)
        image = Image.open(image_path).convert("RGB")

        # Preprocess the image to match LLaVA's expected format
        image_input = tokenizer(images=image, return_tensors="pt").to(device)  # Adjust according to LLaVA's requirements

        # Get the image features
        image_features = model.get_image_features(**image_input)
        image_features = image_features / image_features.norm(dim=-1, keepdim=True)  # Normalize image features

        # Compute cosine similarity between image features and text features
        similarities = (image_features @ text_features.T).squeeze(0)  # Remove batch dimension

        # Get the color with the highest similarity score
        predicted_color_idx = similarities.argmax().item()
        predicted_color = color_prompts[predicted_color_idx].split()[1]  # Extract the color from the prompt

        # Store the ground truth and predicted labels for evaluation
        y_true.append(ground_truth_color)
        y_pred.append(predicted_color)

        # Check if the prediction is correct (since ground truth is "green")
        if predicted_color == ground_truth_color:
            correct_predictions += 1

# Calculate accuracy
accuracy = correct_predictions / total_images * 100
print(f"Accuracy: {accuracy:.2f}%")

# Get unique labels from ground truth and predictions
unique_labels = sorted(list(set(y_true + y_pred)))  # Dynamically detect classes present in data

# Confusion Matrix and Classification Report
print("\nConfusion Matrix:")
conf_matrix = confusion_matrix(y_true, y_pred, labels=unique_labels)
print(conf_matrix)

print("\nClassification Report:")
class_report = classification_report(y_true, y_pred, target_names=unique_labels, labels=unique_labels)
print(class_report)


ValueError: Unrecognized configuration class <class 'transformers.models.llava.configuration_llava.LlavaConfig'> for this kind of AutoModel: AutoModel.
Model type should be one of AlbertConfig, AlignConfig, AltCLIPConfig, ASTConfig, AutoformerConfig, BarkConfig, BartConfig, BeitConfig, BertConfig, BertGenerationConfig, BigBirdConfig, BigBirdPegasusConfig, BioGptConfig, BitConfig, BlenderbotConfig, BlenderbotSmallConfig, BlipConfig, Blip2Config, BloomConfig, BridgeTowerConfig, BrosConfig, CamembertConfig, CanineConfig, ChameleonConfig, ChineseCLIPConfig, ChineseCLIPVisionConfig, ClapConfig, CLIPConfig, CLIPVisionConfig, CLIPSegConfig, ClvpConfig, LlamaConfig, CodeGenConfig, CohereConfig, ConditionalDetrConfig, ConvBertConfig, ConvNextConfig, ConvNextV2Config, CpmAntConfig, CTRLConfig, CvtConfig, Data2VecAudioConfig, Data2VecTextConfig, Data2VecVisionConfig, DbrxConfig, DebertaConfig, DebertaV2Config, DecisionTransformerConfig, DeformableDetrConfig, DeiTConfig, DetaConfig, DetrConfig, Di

In [23]:
import argparse
import os
import subprocess
import torch
from PIL import Image
from transformers import AutoTokenizer, AutoModel
from vllm import LLM
from vllm.multimodal.image import ImageFeatureData, ImagePixelData
from sklearn.metrics import confusion_matrix, classification_report

def run_llava_pixel_values(image_path, llm, tokenizer, color_prompts):
    prompt = "<image>" * 576 + "\nUSER: What color is the car in this image?\nASSISTANT:"

    # Load and preprocess the image
    image = Image.open(image_path).convert("RGB")
    image_pixel_values = tokenizer(images=image, return_tensors="pt")['pixel_values']

    # Generate predictions
    outputs = llm.generate({
        "prompt": prompt,
        "multi_modal_data": ImagePixelData(image_pixel_values),
    })

    # Extract the generated text
    generated_text = outputs[0].outputs[0].text
    return generated_text

def run_llava_image_features(image_path, llm, tokenizer, color_prompts):
    prompt = "<image>" * 576 + "\nUSER: What color is the car in this image?\nASSISTANT:"

    # Load and preprocess the image
    image = Image.open(image_path).convert("RGB")
    image_features = tokenizer(images=image, return_tensors="pt")['image_features']

    # Generate predictions
    outputs = llm.generate({
        "prompt": prompt,
        "multi_modal_data": ImageFeatureData(image_features),
    })

    # Extract the generated text
    generated_text = outputs[0].outputs[0].text
    return generated_text

def main(args):
    llm = LLM(
        model="llava-hf/llava-1.5-7b-hf",
        image_input_type=args.type,
        image_token_id=32000,
        image_input_shape="1,3,336,336" if args.type == "pixel_values" else "1,576,1024",
        image_feature_size=576,
    )
    tokenizer = AutoTokenizer.from_pretrained("llava-hf/llava-1.5-7b-hf")

    image_folder = "images"
    color_prompts = ["a red car", "a blue car", "a green car", "a yellow car", "a black car"]

    y_true = []
    y_pred = []

    # Iterate over each image in the folder
    for image_file in os.listdir(image_folder):
        if image_file.endswith(('png', 'jpg', 'jpeg')):
            image_path = os.path.join(image_folder, image_file)
            ground_truth_color = "green"  # Replace with actual ground truth if known

            # Generate prediction
            if args.type == "pixel_values":
                generated_text = run_llava_pixel_values(image_path, llm, tokenizer, color_prompts)
            else:
                generated_text = run_llava_image_features(image_path, llm, tokenizer, color_prompts)

            # Extract color from the generated text
            predicted_color = generated_text.split()[0]  # Extract the color part from the output

            # Store results for evaluation
            y_true.append(ground_truth_color)
            y_pred.append(predicted_color)

    # Calculate accuracy
    accuracy = (sum([1 for true, pred in zip(y_true, y_pred) if true == pred]) / len(y_true)) * 100
    print(f"Accuracy: {accuracy:.2f}%")

    # Get unique labels from ground truth and predictions
    unique_labels = sorted(list(set(y_true + y_pred)))

    # Confusion Matrix and Classification Report
    print("\nConfusion Matrix:")
    conf_matrix = confusion_matrix(y_true, y_pred, labels=unique_labels)
    print(conf_matrix)

    print("\nClassification Report:")
    class_report = classification_report(y_true, y_pred, target_names=unique_labels, labels=unique_labels)
    print(class_report)

if __name__ == "__main__":
    parser = argparse.ArgumentParser(description="Demo on LLaVA")
    parser.add_argument("--type",
                        type=str,
                        choices=["pixel_values", "image_features"],
                        default="pixel_values",
                        help="image input type")
    args = parser.parse_args()

    # Download from s3
    s3_bucket_path = "s3://air-example-data-2/vllm_opensource_llava/"
    local_directory = "images"

    # Make sure the local directory exists or create it
    os.makedirs(local_directory, exist_ok=True)

    # Use AWS CLI to sync the directory, assume anonymous access
    subprocess.check_call([
        "aws",
        "s3",
        "sync",
        s3_bucket_path,
        local_directory,
        "--no-sign-request",
    ])
    
    main(args)


ImportError: cannot import name 'ImageFeatureData' from 'vllm.multimodal.image' (c:\Users\Asus\Downloads\llm-main\llm-main\.venv\Lib\site-packages\vllm\multimodal\image.py)

In [24]:
import os
from PIL import Image
import torch
from transformers import AutoProcessor, AutoModelForCausalLM
from sklearn.metrics import confusion_matrix, classification_report

# Load the pre-trained LLaVA model and processor
model_name = "llava-hf/llava-1.5-7b-hf"
model = AutoModelForCausalLM.from_pretrained(model_name)
processor = AutoProcessor.from_pretrained(model_name)

# Define the text prompts for different colors
color_prompts = ["a red car", "a blue car", "a green car", "a yellow car", "a black car"]
color_labels = [prompt.split()[1] for prompt in color_prompts]  # Extract color labels from prompts

# Initialize performance metrics
total_images = 0
correct_predictions = 0

# Arrays to store the predicted and actual labels
y_true = []
y_pred = []

# Folder where your car images are stored
image_folder = "path/to/your/images"  # Change this to the folder containing your images

# Process each image
for image_file in os.listdir(image_folder):
    if image_file.endswith(('png', 'jpg', 'jpeg')):
        total_images += 1

        # Load and preprocess the image
        image_path = os.path.join(image_folder, image_file)
        image = Image.open(image_path).convert("RGB")

        # Create conversation template
        conversation = [
            {
                "role": "user",
                "content": [
                    {"type": "image", "image": image},
                    {"type": "text", "text": "What color is the car in this image?"},
                ],
            }
        ]

        # Format the prompt
        text_prompt = processor.apply_chat_template(conversation, add_generation_prompt=True)

        # Tokenize the prompt
        inputs = processor(text=text_prompt, return_tensors="pt")
        pixel_values = processor(images=image, return_tensors="pt").pixel_values

        # Generate predictions
        outputs = model.generate(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'])
        generated_text = processor.decode(outputs[0], skip_special_tokens=True)

        # Extract color from generated text
        predicted_color = generated_text.split()[0].lower()  # Extract color part from the output

        # Append to results
        y_true.append("green")  # Assuming ground truth is "green" for all images
        y_pred.append(predicted_color)

        # Check if the prediction is correct
        if predicted_color == "green":
            correct_predictions += 1

# Calculate accuracy
accuracy = (correct_predictions / total_images) * 100
print(f"Accuracy: {accuracy:.2f}%")

# Confusion Matrix and Classification Report
unique_labels = sorted(list(set(y_true + y_pred)))

print("\nConfusion Matrix:")
conf_matrix = confusion_matrix(y_true, y_pred, labels=unique_labels)
print(conf_matrix)

print("\nClassification Report:")
class_report = classification_report(y_true, y_pred, target_names=unique_labels, labels=unique_labels)
print(class_report)


ValueError: Unrecognized configuration class <class 'transformers.models.llava.configuration_llava.LlavaConfig'> for this kind of AutoModel: AutoModelForCausalLM.
Model type should be one of BartConfig, BertConfig, BertGenerationConfig, BigBirdConfig, BigBirdPegasusConfig, BioGptConfig, BlenderbotConfig, BlenderbotSmallConfig, BloomConfig, CamembertConfig, LlamaConfig, CodeGenConfig, CohereConfig, CpmAntConfig, CTRLConfig, Data2VecTextConfig, DbrxConfig, ElectraConfig, ErnieConfig, FalconConfig, FuyuConfig, GemmaConfig, Gemma2Config, GitConfig, GPT2Config, GPT2Config, GPTBigCodeConfig, GPTNeoConfig, GPTNeoXConfig, GPTNeoXJapaneseConfig, GPTJConfig, JambaConfig, JetMoeConfig, LlamaConfig, MambaConfig, Mamba2Config, MarianConfig, MBartConfig, MegaConfig, MegatronBertConfig, MistralConfig, MixtralConfig, MptConfig, MusicgenConfig, MusicgenMelodyConfig, MvpConfig, NemotronConfig, OlmoConfig, OpenLlamaConfig, OpenAIGPTConfig, OPTConfig, PegasusConfig, PersimmonConfig, PhiConfig, Phi3Config,